In [1]:
% load_ext autoreload
% autoreload 2

In [2]:
!pip install seaborn
!pip install numpy
!pip install pandas
!pip install river
!pip install scikit-learn
!pip install lightgbm
!pip install matplotlib

In [64]:
import warnings

import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from lightgbm import LGBMClassifier
from matplotlib import pyplot as plt
import drift_detector_with_labels, deepcheck_detectors, evidently_ai_detectors, drift,
    drift_detector_multivariate_hdddm, drift_detector_multivariate_md3, drift_detector_multivariate_ollindda

warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [4]:
red_wine_dataset = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv", sep=";")
white_wine_dataset = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv", sep=";")

In [5]:
red_wine_dataset['wine_type'] = "red"
white_wine_dataset['wine_type'] = "white"

In [6]:
wine_dataset = pd.concat([red_wine_dataset, white_wine_dataset], axis=0)
wine_dataset["wine_type"] = np.where(wine_dataset["wine_type"] == "red", 1, 0)
wine_dataset.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,wine_type
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,1
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1


In [7]:
conditions = [
    (wine_dataset["quality"] <= 3),
    (wine_dataset.quality > 3) & (wine_dataset.quality <= 6),
    (wine_dataset["quality"] > 6)
]

values = [0, 1, 2]

wine_dataset['quality'] = np.select(conditions, values)

wine_dataset['quality'].value_counts()

1    5190
2    1277
0      30
Name: quality, dtype: int64

In [8]:
wine_dataset_train, wine_dataset_test = train_test_split(wine_dataset, test_size=0.4, shuffle=True)
X_train = wine_dataset_train.loc[:, wine_dataset_train.columns != "quality"]
y_train = wine_dataset_train["quality"]
X_test = wine_dataset_test.loc[:, wine_dataset_test.columns != "quality"]
y_test = wine_dataset_test["quality"]

In [56]:
drift_detector_labels_gradual_drift(wine_dataset_train, wine_dataset_test, "alcohol", "quality",
                                    2, "increase", 100, 5)

Day 0 :
Drift Detected
Day 1 :
Day 2 :
Drift Detected
Day 3 :
Drift Detected
Day 4 :
Drift Detected
Day 5 :


In [63]:
drift_detector_multivariate_md3.md3_seasonal_drift(wine_dataset_train, wine_dataset_test, "alcohol", "quality",
                                                   2, 3, "increase", 100, 10)

Day 0 :
Alarm, drift Detected
Day 1 :
Alarm, drift Detected
Day 2 :
Day 3 :
Alarm, drift Detected
Day 4 :
Alarm, drift Detected
Day 5 :
Day 6 :
Alarm, drift Detected
Day 7 :
Alarm, drift Detected
Day 8 :
Day 9 :
Alarm, drift Detected
Day 10 :
Alarm, drift Detected


In [71]:
drift_detector_multivariate_ollindda.olindda_gradual_drift(data_train= wine_dataset_train,
                                                           data_to_compare=wine_dataset_test,
                                                           column_name = "alcohol",
                                                            n_clusters = 6,
                                                           value_drift = 2,
                                                           action = "increase",
                                                           nb_sample = 100,
                                                           nb_days = 5)

Day 0 :
    fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0             7.4              0.50         0.47            2.00      0.086   
1             8.2              1.00         0.09            2.30      0.065   
2             6.9              0.24         0.39            1.30      0.063   
3             6.4              0.34         0.20           14.90      0.060   
4             6.3              0.15         0.30            1.40      0.022   
..            ...               ...          ...             ...        ...   
95            6.9              0.25         0.27            9.05      0.039   
96            7.2              0.21         0.34           11.90      0.043   
97            7.1              0.31         0.25           11.20      0.048   
98            6.6              0.29         0.29            1.80      0.036   
99            7.1              0.24         0.41           17.80      0.046   

    free sulfur dioxide  total sulfur dioxi

AttributeError: 'NoneType' object has no attribute 'split'